In [260]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import re
from bs4 import BeautifulSoup as bs
import os 
import glob
import time
import re
from tqdm.notebook import tqdm, trange

## COMBIEN MULTPLE FILES INTO ONE

In [ ]:
path = r'C:\Users\Dell\Documents\Data powerBi\Dataset\New folder\New folder'
all_files = glob.glob(os.path.join(path,"*.csv"))
df = pd.concat((pd.read_csv(f) for f in all_files))
df

df.to_csv('jobs_full.csv', index= False)

In [293]:
data = pd.read_csv('jobs_full.csv')

## location

In [294]:
data['location'] = data['location'].str.replace('Tout le Maroc','Tout le Maroc / A distance')
unique_elements = list(set(data['location'].unique()))

sorted_unique_elements = sorted(unique_elements)

print(sorted_unique_elements)

['Agadir ', 'Al hoceima ', 'Assilah ', 'Autre ville ', 'Azrou ', 'Benguerir ', 'Beni Mellal ', 'Benslimane ', 'Berkane ', 'Berrechid ', 'Boujdour ', 'Bouskoura ', 'Bouznika ', 'Casablanca ', 'Chafchaouen ', 'Dakhla ', 'Deroua ', 'El jadida ', 'Errachidia ', 'Essaouira ', 'Essmara ', 'Fkih Ben Salah ', 'Fès ', 'Guelmim ', 'Ifrane ', 'Imouzzer Kandar ', 'Kabila ', 'Khemisset ', 'Khenifra ', 'Khouribga ', 'Kénitra ', 'Laayoune ', 'Larache ', 'Marrakech ', 'Martil ', 'Mediouna ', 'Meknès ', 'Midelt ', 'Mohammedia ', 'Nador ', 'Nouaceur ', 'Oualidia ', 'Ouarzazate ', 'Ouazzane ', 'Oujda ', 'Rabat ', 'Safi ', 'Saidia ', 'Salé ', 'Sefrou ', 'Settat ', 'Sidi Bennour ', 'Sidi Rahal ', 'Sidi Slimane ', 'Sidi kacem ', 'Souk Elarbaa Du Gharb ', 'Tamensourte ', 'Tamesna ', 'Tanger ', 'Tarfaya ', 'Taroudant ', 'Taza ', 'Temara ', 'Tetouan ', 'Tifelt ', 'Tinghir ', 'Tiznit ', 'Tout le Maroc / A distance ', 'Youssoufia ', 'Zagora ', 'ksar el kebir ']


## more_info

In [295]:
data['more_info'].str.split(':', expand = True).head()

,0,1,2,3,4,5,6,7
0,Domaine,Services à La Personne Fonction,Cuisinier/Serveur(se)/Femme ménage/Nounou Con...,A discuter Entreprise,Sabim shop Salaire,A discuter Niveau d'études,Niveau Bac,None
1,Domaine,Commercial / Vente Fonction,Commercial Vendeur magasin/Showroom Contrat,A discuter Entreprise,Confidentiel Salaire,A discuter Niveau d'études,Bac plus 2,None
2,Domaine,Banque / Finance / Assurance Fonction,Commercial particuliers (B2C) Contrat,CDI Entreprise,Prive Salaire,A discuter Niveau d'études,Bac plus 2,None
3,Domaine,Tourisme / Hôtellerie / Restauration Fonction,Assistant/Ventes/Accueil Contrat,CDI Entreprise,confidentiel Salaire,A discuter Niveau d'études,Bac plus 3,None
4,Domaine,Immobilier Fonction,Commercial professionnels (B2B) Contrat,CDI Entreprise,FIRST ONE IMMOBILIER Salaire,A discuter Niveau d'études,Bac plus 3,None


In [296]:
data['domaine'] = data['more_info'].str.split(':', expand = True)[1]
data['contrat'] = data['more_info'].str.split(':', expand = True)[3]
data['salaire'] = data['more_info'].str.split(':', expand = True)[5]
data['etude'] = data['more_info'].str.split(':', expand = True)[6]
data['domaine'] = data['domaine'].apply(lambda x: ' '.join(x.split()[:-1]))
data['contrat'] = data['contrat'].apply(lambda x: ' '.join(x.split()[:-1]))
data['salaire'] = data['salaire'].apply(lambda x: ' '.join(x.split()[:-2]))
data.drop(columns=['Unnamed: 0'],inplace=True)
data.drop(columns=['more_info'],inplace=True)

## etude

In [297]:
data['etude'] = data['etude'].str.replace(r'Not important','Pas important').str.strip()
data['etude'] = data['etude'].str.replace(r'البكالوريا بلس 2','Bac plus 2').str.strip()
data['etude'] = data['etude'].str.replace(r'البكالوريا بلس 3','Bac plus 3').str.strip()
data['etude'] = data['etude'].str.replace(r'البكالوريا بلس 4','Bac plus 4').str.strip()
data['etude'] = data['etude'].str.replace(r'البكالوريا بلس 5','Bac plus 5').str.strip()
data['etude'] = data['etude'].str.replace(r'البكالوريا','Bac').str.strip()
data['etude'] = data['etude'].str.replace(r'Bac','Bac').str.strip()
data['etude'] = data['etude'].str.replace(r'ليس مهم','Pas important').str.strip()
data['etude'] = data['etude'].str.replace(r'ليس مهما','Pas important').str.strip()

In [298]:
values_to_keep = ['Bac plus 2', 'Pas important', 'Bac plus 3', 'Bac plus 5', 'Niveau Bac', 'Bac plus 4','Bac']
data = data[data['etude'].isin(values_to_keep)]
data['etude'].value_counts()

Bac plus 2       8541
Pas important    4610
Bac plus 3       4224
Bac plus 5       2670
Niveau Bac       1662
Bac plus 4        909
Name: etude, dtype: int64

## salaire

In [299]:
data['salaire'] = data['salaire'].str.replace(r'To be discussed','A discuter').str.strip()
data['salaire'] = data['salaire'].str.replace(r'To discuss','A discuter').str.strip()
data['salaire'] = data['salaire'].str.replace(r'يناقش','A discuter').str.strip()

In [300]:
values_to_salaire = [r'A discuter', '2 000 - 3 000 DH', '3 000 - 4 000 DH', '4 000 - 6 000 DH', '6 000 - 8 000 DH', '8 000 - 10 000 DH','Plus de 10 000 DH','Plus de 20 000 DH','Plus de 30 000 DH','Commission']
data = data[data['salaire'].isin(values_to_salaire)]
data['salaire'].value_counts()

A discuter           17327
3 000 - 4 000 DH      1658
2 000 - 3 000 DH      1526
4 000 - 6 000 DH       999
6 000 - 8 000 DH       354
Commission             350
Plus de 10 000 DH      228
8 000 - 10 000 DH      129
Plus de 20 000 DH       35
Plus de 30 000 DH        8
Name: salaire, dtype: int64

## contrat

In [301]:
data['contrat'] = data['contrat'].str.replace(r'يناقش','A discuter')
data['contrat'] = data['contrat'].str.replace(r'To discuss','A discuter')
data['contrat'] = data['contrat'].str.replace(r'Internship','Stage')

In [302]:
values_to_contrat = [r'A discuter', 'CDI' ,'Interim' ,'Stage', 'CDD' ,'Anapec']
data = data[data['contrat'].isin(values_to_contrat)]
data['contrat'].value_counts()

A discuter    9657
CDI           7693
Interim       1768
Stage         1607
CDD           1283
Anapec         606
Name: contrat, dtype: int64

## domaine

In [303]:
pattern_french = re.compile(r'[A-Za-zÀ-ÿ\s]+')

data = data[data['domaine'].str.match(pattern_french, na=False)]

data['domaine'].value_counts()

Tourisme / Hôtellerie / Restauration            3322
Commercial / Vente                              2377
Industrie / Ingénierie / Energie                1686
Autre                                           1588
Education / Social / Petite Enfance             1515
Accueil / Secrétariat / Administration          1453
Informatique / Multimédia / Internet            1233
Construction / Btp                              1180
Comptabilité / Gestion / Audit                  1156
Marketing / Communication / Publicité / Rp       779
Transport / Achat / Logistique                   686
Santé / Médical                                  619
Banque / Finance / Assurance                     597
Grande Distribution                              566
Automobile                                       456
Maintenance / Entretien                          454
Ressources Humaines / Recrutement / Intérim      369
Sécurité / Surveillance / Gardiennage            337
Immobilier                                    

## experiences

In [305]:
data.loc[data['experience'].str.contains(r'2 mois|2 Mois|2mois', case=False, na=False), 'experience'] = "2 mois"
data.loc[data['experience'].str.contains(r'3 mois|3 Mois|3mois', case=False, na=False), 'experience'] = "3 mois"
data.loc[data['experience'].str.contains(r'4 mois|4 Mois|4mois', case=False, na=False), 'experience'] = "4 mois"
data.loc[data['experience'].str.contains(r'5 mois|5 Mois|5mois', case=False, na=False), 'experience'] = "5 mois"
data.loc[data['experience'].str.contains(r'6 mois|6 Mois|6mois', case=False, na=False), 'experience'] = "6 mois"
data.loc[data['experience'].str.contains(r'un an|1 an|1an', case=False, na=False), 'experience'] = "1 ans"
data.loc[data['experience'].str.contains(r'2 ans| 2ans', case=False, na=False), 'experience'] = "2 ans"
data.loc[data['experience'].str.contains(r'3 ans| 3ans', case=False, na=False), 'experience'] = "3 ans"
data.loc[data['experience'].str.contains(r'4 ans| 4ans', case=False, na=False), 'experience'] = "4 ans"
data.loc[data['experience'].str.contains(r'5 ans| 5ans', case=False, na=False), 'experience'] = "5 ans"
data.loc[data['experience'].str.contains(r'6 ans| 6ans', case=False, na=False), 'experience'] = "6 ans"
data.loc[data['experience'].str.contains(r'7 ans| 7ans', case=False, na=False), 'experience'] = "7 ans"
data.loc[data['experience'].str.contains(r'8 ans| 8ans', case=False, na=False), 'experience'] = "8 ans"
data.loc[data['experience'].str.contains(r'9 ans| 9ans', case=False, na=False), 'experience'] = "9 ans"
data.loc[data['experience'].str.contains(r'10 ans| 10ans', case=False, na=False), 'experience'] = "10 ans"
data.loc[data['experience'].str.contains(r'No experience|Non experience', case=False, na=False), 'experience'] = "Pas neccessaire"
data.loc[data['experience'].str.contains(r"expérience|Expérience|Experience|l’EXPÉRIENCE|EXPERIENCE|experience|expérimentées|l'expérience|d'expérience|d'expérience", case=False, na=False), 'experience'] = "Deja Expérience/expérimentées/Premiere experiences "


In [306]:
data['experience'].value_counts()

Pas neccessaire                                        11277
Deja Expérience/expérimentées/Premiere experiences      6978
2 ans                                                   1331
5 ans                                                    992
3 ans                                                    959
1 ans                                                    508
4 ans                                                    189
10 ans                                                   136
6 mois                                                    82
6 ans                                                     60
8 ans                                                     36
7 ans                                                     35
3 mois                                                    17
2 mois                                                    10
Name: experience, dtype: int64

## titles 

In [307]:
jobs = data.loc[~data['titles'].str.contains(r'stag|Stag|STAG')]

In [308]:
# Regular expression for French text
pattern_french = re.compile(r'[A-Za-zÀ-ÿ\s]+')

# Filtering non-French rows
jobs = jobs[jobs['titles'].str.match(pattern_french, na=False)]


In [313]:
jobs.loc[(jobs['titles'].str.contains(r'assis', case=False, na=False)) & (jobs['titles'].str.contains(r'admini', case=False, na=False)), 'titles'] = "Assistant(e) administrative"
jobs.loc[(jobs['titles'].str.contains(r'assis', case=False, na=False)) & (jobs['titles'].str.contains(r'direc', case=False, na=False)), 'titles'] = "Assistant(e) de direction"
jobs.loc[(jobs['titles'].str.contains(r'assis', case=False, na=False)) & (jobs['titles'].str.contains(r'polyv', case=False, na=False)), 'titles'] = "Assistant(e) polyvalente "
jobs.loc[jobs['titles'].str.contains(r'gestion ', case=False, na=False), 'titles'] = "Assistant(e) de gestion"
jobs.loc[jobs['titles'].str.contains(r'gestionnaire', case=False, na=False), 'titles'] = "Gestionnaire"
jobs.loc[jobs['titles'].str.contains(r'achete|achat', case=False, na=False), 'titles'] = "Acheteur/Responsable achat/Assistant(e) achat"
jobs.loc[jobs['titles'].str.contains(r'prof |professeur', case=False, na=False), 'titles'] = "Professeur"
jobs.loc[jobs['titles'].str.contains(r'formateur|formatrice', case=False, na=False), 'titles'] = "Formateur/Formatrice"
jobs.loc[jobs['titles'].str.contains(r'Maîtresse|Maitresse|Maître|Maitre|educat|éducat|enseignant|enseignant(e)|Enseig|ENSEINGNANTS', case=False, na=False), 'titles'] = "Maitre(sse)/educatrice/Enseignante"
jobs.loc[jobs['titles'].str.contains(r'juri', case=False, na=False), 'titles'] = "Assistant(e) juridice/Juriste"
jobs.loc[jobs['titles'].str.contains(r'avocat', case=False, na=False), 'titles'] = "Avocat"
jobs.loc[jobs['titles'].str.contains(r'commerc|comerc', case=False, na=False), 'titles'] = "Commercial(e)/ assistant(e) commerciale(e)"
jobs.loc[jobs['titles'].str.contains(r'compta|comtable|compat', case=False, na=False), 'titles'] = "Compatble/ assistant(e) comptable"
jobs.loc[jobs['titles'].str.contains(r'infog|design|Graphiste|graph|webdesig', case=False, na=False), 'titles'] = "Infographiste/web designer/Graphiste"
jobs.loc[jobs['titles'].str.contains(r'community', case=False, na=False), 'titles'] = "Community manager"
jobs.loc[jobs['titles'].str.contains(r'communication', case=False, na=False), 'titles'] = "Agent de communication"
jobs.loc[jobs['titles'].str.contains(r'developpeur|développeur|programmeur|python|java|Fullstack|php|js',case=False, na=False), 'titles'] = "Développeur Web"
jobs.loc[jobs['titles'].str.contains(r'marketin|Webmarkteur|markete', case=False, na=False), 'titles'] = "Marketing digital/Web-markteur"
jobs.loc[jobs['titles'].str.contains(r'ads|seo', case=False, na=False), 'titles'] = "Ads & SEO expert"
jobs.loc[jobs['titles'].str.contains(r'E-commerce|ecommerce|dropshipping', case=False, na=False), 'titles'] = "E-commerce/dropshipping Spécialiste "
jobs.loc[jobs['titles'].str.contains(r'video|Vidéaste|Vidéo', case=False, na=False), 'titles'] = "video montage/Vidéaste/Videography"
jobs.loc[jobs['titles'].str.contains(r'vend', case=False, na=False), 'titles'] = "vendeur/vendeuse"
jobs.loc[jobs['titles'].str.contains(r'RH|humain|HR ', case=False, na=False), 'titles'] = "RH/ assistant(e) RH"
jobs.loc[jobs['titles'].str.contains(r'civil|genie |génie ', case=False, na=False), 'titles'] = "Ingénieur Génie Civil/Technicien génie civile"
jobs.loc[jobs['titles'].str.contains(r'logisti|transpo', case=False, na=False), 'titles'] = "Logistique et transport"
jobs.loc[jobs['titles'].str.contains(r'touris|tourisme', case=False, na=False), 'titles'] = "agent de tourisme"
jobs.loc[jobs['titles'].str.contains(r'secret|secrét', case=False, na=False), 'titles'] = "secrétaire"
jobs.loc[jobs['titles'].str.contains(r'pharmacienne|pharmaci', case=False, na=False), 'titles'] = "pharmacienne"
jobs.loc[jobs['titles'].str.contains(r'dentaire', case=False, na=False), 'titles'] = "assistant(e) dentaire"
jobs.loc[jobs['titles'].str.contains(r'standar', case=False, na=False), 'titles'] = "assistant(e) standariste"
jobs.loc[jobs['titles'].str.contains(r'livre', case=False, na=False), 'titles'] = "livreur"
jobs.loc[jobs['titles'].str.contains(r'serve', case=False, na=False), 'titles'] = "serveur/serveuse"
jobs.loc[jobs['titles'].str.contains(r'saisie', case=False, na=False), 'titles'] = "Opératreur(ice) de saisie"
jobs.loc[jobs['titles'].str.contains(r'operation|opération', case=False, na=False), 'titles'] = "Agent d'opération"
jobs.loc[jobs['titles'].str.contains(r'assuran', case=False, na=False), 'titles'] = "assitant(e) en assurance"
jobs.loc[jobs['titles'].str.contains(r'maintenance', case=False, na=False), 'titles'] = "Technicien en Maintenance"
jobs.loc[jobs['titles'].str.contains(r'accueil|acceuil', case=False, na=False), 'titles'] = "Agent d'accueil/Hotesse d'acceuil"
jobs.loc[jobs['titles'].str.contains(r'électricien|electricien|electro|électro|Electricité|éléctr', case=False, na=False), 'titles'] = "electricien/Chargé en Electricité"
jobs.loc[jobs['titles'].str.contains(r'cuisin', case=False, na=False), 'titles'] = "cuisinier"
jobs.loc[jobs['titles'].str.contains(r'mecanqiue|mécanique|Mécanicien|mecani', case=False, na=False), 'titles'] = "fabrication mécanique/Mécanicien"
jobs.loc[jobs['titles'].str.contains(r'Conseiller|conseill|clientèle|clientele|client', case=False, na=False), 'titles'] = "Conseiller/Chargé clientèle"
jobs.loc[jobs['titles'].str.contains(r'portefeuil|portefeuille', case=False, na=False), 'titles'] = "chargé portefeuille"
jobs.loc[jobs['titles'].str.contains(r'recouvrement', case=False, na=False), 'titles'] = "chargés de recouvrement"
jobs.loc[jobs['titles'].str.contains(r'Barman|bar', case=False, na=False), 'titles'] = "Barman"
jobs.loc[jobs['titles'].str.contains(r'menage|ménage|chambre|NETTOYAGE|nettoyage', case=False, na=False), 'titles'] = "femme de ménage"
jobs.loc[jobs['titles'].str.contains(r'Magasinier|magas', case=False, na=False), 'titles'] = "Magasinier"
jobs.loc[jobs['titles'].str.contains(r'coiff', case=False, na=False), 'titles'] = "Coiffeuse"
jobs.loc[jobs['titles'].str.contains(r'rang|RANG', case=False, na=False), 'titles'] = "Chef de rang/ commis de rang"
jobs.loc[jobs['titles'].str.contains(r'Auxiliaire de vie|Auxiliaire|vie', case=False, na=False), 'titles'] = "Auxiliaire de vie"
jobs.loc[jobs['titles'].str.contains(r'gerant|gérant', case=False, na=False), 'titles'] = "Gérant"
jobs.loc[jobs['titles'].str.contains(r'cable', case=False, na=False), 'titles'] = "cableur"
jobs.loc[jobs['titles'].str.contains(r'directr|directe', case=False, na=False), 'titles'] = "Directeur/Directrice"
jobs.loc[jobs['titles'].str.contains(r'Chauffeur|CHAUFEUR', case=False, na=False), 'titles'] = "Chauffeur"
jobs.loc[jobs['titles'].str.contains(r'plomb', case=False, na=False), 'titles'] = "Plombier"
jobs.loc[jobs['titles'].str.contains(r'caiss', case=False, na=False), 'titles'] = "Caissier/Caissière"
jobs.loc[jobs['titles'].str.contains(r'model|modeliste|modél', case=False, na=False), 'titles'] = "Model"
jobs.loc[jobs['titles'].str.contains(r'consult', case=False, na=False), 'titles'] = "Consultant"
jobs.loc[jobs['titles'].str.contains(r'contro|Contrô', case=False, na=False), 'titles'] = "Contrôleur"
jobs.loc[jobs['titles'].str.contains(r'chantier', case=False, na=False), 'titles'] = "Chef de chantier"
jobs.loc[jobs['titles'].str.contains(r'projet ', case=False, na=False), 'titles'] = "Chef de projet"
jobs.loc[jobs['titles'].str.contains(r'partie', case=False, na=False), 'titles'] = "Chef de partie"
jobs.loc[jobs['titles'].str.contains(r'équipe|equipe', case=False, na=False), 'titles'] = "chef d'equipe"
jobs.loc[jobs['titles'].str.contains(r'recept|récep', case=False, na=False), 'titles'] = "Agent de reception/Receptionniste"
jobs.loc[jobs['titles'].str.contains(r'vent', case=False, na=False), 'titles'] = "Responsable/Administrateur des ventes"
jobs.loc[jobs['titles'].str.contains(r'office', case=False, na=False), 'titles'] = "Front office/Back office/Office manager"
jobs.loc[jobs['titles'].str.contains(r'aide', case=False, na=False), 'titles'] = "Aide general(e)"
jobs.loc[jobs['titles'].str.contains(r'sushi', case=False, na=False), 'titles'] = "sushiman"
jobs.loc[jobs['titles'].str.contains(r'pizza', case=False, na=False), 'titles'] = "Pizzaiolo"
jobs.loc[jobs['titles'].str.contains(r'patis', case=False, na=False), 'titles'] = "Patissier/patissière"
jobs.loc[jobs['titles'].str.contains(r'Boulan', case=False, na=False), 'titles'] =   "Boulangerie"
jobs.loc[jobs['titles'].str.contains(r'spa ', case=False, na=False), 'titles'] = "Spa thérapeute"
jobs.loc[jobs['titles'].str.contains(r'recept|récept', case=False, na=False), 'titles'] = "Réceptionniste"
jobs.loc[jobs['titles'].str.contains(r'mede|méde', case=False, na=False), 'titles'] = "médcin(e)"
jobs.loc[jobs['titles'].str.contains(r'Conducteur', case=False, na=False), 'titles'] = "Conducteur de Travaux"
jobs.loc[jobs['titles'].str.contains(r'dessin', case=False, na=False), 'titles'] = "Dessinateur/Projeteur/2D-3D"
jobs.loc[jobs['titles'].str.contains(r'Infirmi', case=False, na=False), 'titles'] = "Infirmière"
jobs.loc[jobs['titles'].str.contains(r'menuis', case=False, na=False), 'titles'] = "Menuisier"
jobs.loc[jobs['titles'].str.contains(r'plong', case=False, na=False), 'titles'] = "Plongeur"
jobs.loc[jobs['titles'].str.contains(r'soud', case=False, na=False), 'titles'] = "Soudeur"
jobs.loc[jobs['titles'].str.contains(r'pompi', case=False, na=False), 'titles'] = "Pompiste"
jobs.loc[jobs['titles'].str.contains(r'tourn', case=False, na=False), 'titles'] = "Tourneur"
jobs.loc[jobs['titles'].str.contains(r'factur', case=False, na=False), 'titles'] = "Agent de Facturation"
jobs.loc[jobs['titles'].str.contains(r'import|export', case=False, na=False), 'titles'] = "assistant(e)/agent import-export"
jobs.loc[jobs['titles'].str.contains(r'Délégué|delegue', case=False, na=False), 'titles'] = "Délégué"
jobs.loc[jobs['titles'].str.contains(r'gardie', case=False, na=False), 'titles'] = "Agent de gardiennage"
jobs.loc[jobs['titles'].str.contains(r'surveill|surveillant', case=False, na=False), 'titles'] = "Surveillant/Video surveillance"
jobs.loc[jobs['titles'].str.contains(r'data', case=False, na=False), 'titles'] = "Data manager"
jobs.loc[jobs['titles'].str.contains(r'Business analyst|analyst', case=False, na=False), 'titles'] = "Business analyst"
jobs.loc[jobs['titles'].str.contains(r'animate', case=False, na=False), 'titles'] = "Animateur"
jobs.loc[jobs['titles'].str.contains(r'Déclarant|declarant', case=False, na=False), 'titles'] = "Déclarant en douane "
jobs.loc[jobs['titles'].str.contains(r'distribu', case=False, na=False), 'titles'] = "Distributeur"
jobs.loc[jobs['titles'].str.contains(r'metere|métre', case=False, na=False), 'titles'] = "Métreur"
jobs.loc[jobs['titles'].str.contains(r'coursi', case=False, na=False), 'titles'] = "Coursier"
jobs.loc[jobs['titles'].str.contains(r'prepa|Prép', case=False, na=False), 'titles'] = "Preparateur de commande"
jobs.loc[jobs['titles'].str.contains(r'archi', case=False, na=False), 'titles'] = "Archiviste"
jobs.loc[jobs['titles'].str.contains(r'enquêt', case=False, na=False), 'titles'] = "enquêteur"
jobs.loc[jobs['titles'].str.contains(r'planification|plani', case=False, na=False), 'titles'] = "Planificateur/Ingénieur planification "
jobs.loc[jobs['titles'].str.contains(r'jardi', case=False, na=False), 'titles'] = "Jardinier"
jobs.loc[jobs['titles'].str.contains(r'tradu', case=False, na=False), 'titles'] = "Traducteur"
jobs.loc[jobs['titles'].str.contains(r'réservation|réserv|reserv', case=False, na=False), 'titles'] = "Agent de réservation"
jobs.loc[jobs['titles'].str.contains(r'ouvrie', case=False, na=False), 'titles'] ="Ouvrier"
jobs.loc[jobs['titles'].str.contains(r'lavage', case=False, na=False), 'titles'] = "Lavage"
jobs.loc[jobs['titles'].str.contains(r'audi', case=False, na=False), 'titles'] = "Audite"
jobs.loc[jobs['titles'].str.contains(r'Coach|fitnes', case=False, na=False), 'titles'] ="Coach sportif"
jobs.loc[jobs['titles'].str.contains(r'sécurité|secu|sécu', case=False, na=False), 'titles'] ="Agent de sécurité"
jobs.loc[jobs['titles'].str.contains(r'food|brunch|crep|crêp|Shawarma|cake|gateau|snak', case=False, na=False), 'titles'] = "Fast food/Gateux/crepe"
jobs.loc[jobs['titles'].str.contains(r'coord', case=False, na=False), 'titles'] = "Coordinateur pédagogique/Business"
jobs.loc[jobs['titles'].str.contains(r'qualit', case=False, na=False), 'titles'] = "Responsable qualité/Ingénieur de Qualité"
jobs.loc[jobs['titles'].str.contains(r'produit', case=False, na=False), 'titles'] = "Chef de produit"
jobs.loc[jobs['titles'].str.contains(r'production', case=False, na=False), 'titles'] ='Responsable production/Assistant(e) Production'
jobs.loc[jobs['titles'].str.contains(r'cadr', case=False, na=False), 'titles'] = "Cadre"
jobs.loc[jobs['titles'].str.contains(r'Superviseur', case=False, na=False), 'titles'] = "Superviseur"
jobs.loc[jobs['titles'].str.contains(r'Peint', case=False, na=False), 'titles'] = "Peintre"
jobs.loc[jobs['titles'].str.contains(r'HSE|QHSE', case=False, na=False), 'titles'] = "HSE/QHSE"
jobs.loc[jobs['titles'].str.contains(r'exploitation', case=False, na=False), 'titles'] = "Agent d'exploitation"
jobs.loc[jobs['titles'].str.contains(r'restauran', case=False, na=False), 'titles'] = "Manager Restaurant/Responsable Restaurant"
jobs.loc[jobs['titles'].str.contains(r'Account', case=False, na=False), 'titles'] = "Account Manager"
jobs.loc[jobs['titles'].str.contains(r'bure|Administratif', case=False, na=False), 'titles'] = "Agent de bureau/Agent Administratif"
jobs.loc[jobs['titles'].str.contains(r'adv', case=False, na=False), 'titles'] = "Chargé ADV"
jobs.loc[jobs['titles'].str.contains(r'showroom', case=False, na=False), 'titles'] = "Responsable showroom"
jobs.loc[jobs['titles'].str.contains(r'stock', case=False, na=False), 'titles'] = "Responsable de stock/Gestionnaire de stock"
jobs.loc[jobs['titles'].str.contains(r'Financ', case=False, na=False), 'titles'] = "Responsable financier"
jobs.loc[jobs['titles'].str.contains(r'voyage', case=False, na=False), 'titles'] = "Agent de voyage"
jobs.loc[jobs['titles'].str.contains(' IT|IT ', case=False, na=False), 'titles'] = "IT"
jobs.loc[(jobs['titles'].str.contains(r'Technicien|Téchnicien|informa', case=False, na=False)) & (jobs['titles'].str.contains(r'informa|Réseaux|Reseaux', case=False, na=False)), 'titles'] = "Technicien Réseaux Informatique"
jobs.loc[jobs['titles'].str.contains(r'help|desk', case=False, na=False), 'titles'] = "Help desk"
jobs.loc[jobs['titles'].str.contains(r'Technico commercial|technico-commercial', case=False, na=False), 'titles'] = 'Technico-commercial'
jobs.loc[jobs['titles'].str.contains(r'froid|climat', case=False, na=False), 'titles'] = "Technicien de climatisation"
jobs.loc[jobs['titles'].str.contains(r'contenu', case=False, na=False), 'titles']= "Creator de contenu"


C:\Users\Dell\AppData\Local\Temp\ipykernel_13976\3529171738.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  jobs.loc[jobs['titles'].str.contains(r'Maîtresse|Maitresse|Maître|Maitre|educat|éducat|enseignant|enseignant(e)|Enseig|ENSEINGNANTS', case=False, na=False), 'titles'] = "Maitre(sse)/educatrice/Enseignante"


In [ ]:
jobs['titles'].value_counts().head(50)

In [314]:
jobs.to_csv('jobs-Finalyyy.csv', index= False)